In [1]:
#### network.ipynb
# A module to implement the stochastic gradient descent learning algorithm for a feedforward neural network.
# Gradients are calculated using backpropagation.  Note that I have focused on making the code simple, easily readable, and easily modifiable.

In [2]:
#### Libraries
# Standard library
import random

In [3]:
# Third-party libraries
import numpy as np

In [4]:
class Network(object):

    def __init__(self, sizes):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""
        
        if test_data: 
            n_test = len(test_data)
        n = len(training_data)
        for j in range(epochs):
            np.random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                overall_accuracy, per_digit_accuracy = self.evaluate(test_data)
                print(f"Epoch {j}: {overall_accuracy} / {n_test} correctly classified")
                for digit in range(10):
                    correct, total = per_digit_accuracy[digit]
                    rate = (correct / total * 100) if total > 0 else 0
                    print(f"    Digit {digit}: {rate:.2f}% correct ({correct}/{total})")
            else:
                print(f"Epoch {j} complete")


    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)
    
    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        
        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data]
        overall_correct = sum(int(x == y) for (x, y) in test_results)
        
        per_digit = {i: [0, 0] for i in range(10)}  # {digit: [correct, total]}
        for (x, y) in test_results:
            if x == y:
                per_digit[y][0] += 1  # correct predictions
            per_digit[y][1] += 1      # total predictions for this digit
        return overall_correct, per_digit



    def cost_derivative(self, output_activations, y):
        # Return the vector of partial derivatives \partial C_x / \partial a for the output activations.
        return (output_activations-y)

In [5]:
#### Miscellaneous functions
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

In [6]:
def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

In [7]:
# mnist_loader
# A library to load the MNIST image data.  For details of the data structures that are returned, see the doc strings for ``load_data``
# and ``load_data_wrapper``.  In practice, ``load_data_wrapper`` is the function usually called by our neural network code.

In [8]:
#### Libraries
# Standard library
import pickle
import gzip

In [9]:
# Third-party libraries
import numpy as np

In [10]:
def load_data():
    # Return the MNIST data as a tuple containing the training data, the validation data, and the test data.

    # The ``training_data`` is returned as a tuple with two entries. The first entry contains the actual training images. This is a numpy ndarray 
    # with 50,000 entries. Each entry is, in turn, a numpy ndarray with 784 values, representing the 28 * 28 = 784 pixels in a single MNIST image.

    # The second entry in the ``training_data`` tuple is a numpy ndarray containing 50,000 entries.  Those entries are just the digit values
    # (0...9) for the corresponding images contained in the first entry of the tuple.

    # The ``validation_data`` and ``test_data`` are similar, except ach contains only 10,000 images.

    # This is a nice data format, but for use in neural networks it's helpful to modify the format of the ``training_data`` a little.
    # That's done in the wrapper function ``load_data_wrapper()``, see below.

    f = gzip.open('mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = pickle.load(f, encoding="latin1")
    f.close()
    return (training_data, validation_data, test_data)
    

In [11]:
def load_data_wrapper():
    # Return a tuple containing ``(training_data, validation_data, test_data)``. Based on ``load_data``,
    # but the format is more convenient for use in our implementation of neural networks.

    # In particular, ``training_data`` is a list containing 50,000 2-tuples ``(x, y)``.  ``x`` is a 784-dimensional numpy.ndarray containing 
    # the input image. ``y`` is a 10-dimensional numpy.ndarray representing the unit vector corresponding to the correct digit for ``x``.

    # ``validation_data`` and ``test_data`` are lists containing 10,000 2-tuples ``(x, y)``.  In each case, ``x`` is a 784-dimensional numpy.ndarry
    # containing the input image, and ``y`` is the corresponding classification, i.e., the digit values (integers) corresponding to ``x``.

    # Obviously, this means we're using slightly different formats for the training data and the validation / test data.  These formats
    # turn out to be the most convenient for use in our neural network code
    
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = list(zip(training_inputs, training_results))
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = list(zip(validation_inputs, va_d[1]))
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = list(zip(test_inputs, te_d[1]))
    return (training_data, validation_data, test_data)

In [12]:
def vectorized_result(j):
    # Return a 10-dimensional unit vector with a 1.0 in the jth position and zeroes elsewhere. 
    # This is used to convert a digit (0...9) into a corresponding desired output from the neural network.
    
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

In [13]:
def run_experiments():

    # Run the experiments described in chapter 1 of the book.

    training_data, validation_data, test_data = load_data_wrapper()
    net = Network([784 , 30, 10])
    net.SGD(training_data , 30, 10, 3.0, test_data = test_data)

In [14]:
# It is time to call the function run_experiments() to execute all the commands

run_experiments()

Epoch 0: 9081 / 10000 correctly classified
    Digit 0: 97.14% correct (952/980)
    Digit 1: 97.27% correct (1104/1135)
    Digit 2: 90.21% correct (931/1032)
    Digit 3: 90.89% correct (918/1010)
    Digit 4: 92.16% correct (905/982)
    Digit 5: 79.37% correct (708/892)
    Digit 6: 91.75% correct (879/958)
    Digit 7: 90.66% correct (932/1028)
    Digit 8: 86.65% correct (844/974)
    Digit 9: 89.99% correct (908/1009)
Epoch 1: 9241 / 10000 correctly classified
    Digit 0: 97.65% correct (957/980)
    Digit 1: 97.53% correct (1107/1135)
    Digit 2: 89.44% correct (923/1032)
    Digit 3: 93.07% correct (940/1010)
    Digit 4: 94.09% correct (924/982)
    Digit 5: 83.52% correct (745/892)
    Digit 6: 94.15% correct (902/958)
    Digit 7: 93.09% correct (957/1028)
    Digit 8: 91.48% correct (891/974)
    Digit 9: 88.70% correct (895/1009)
Epoch 2: 9304 / 10000 correctly classified
    Digit 0: 96.53% correct (946/980)
    Digit 1: 98.59% correct (1119/1135)
    Digit 2: 93.80% c